In [6]:
#!pip install langchain_community

In [7]:
#!pip install langchain_openai

In [9]:
#!pip install wikipedia

In [10]:
import requests
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Initialize the Wikipedia API Wrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1)

# Define a function to call Wikipedia with limited output
def wikipedia_caller(query: str) -> str:
    """This function queries Wikipedia and returns a condensed summary."""
    full_summary = api_wrapper.run(query)

    # Extract the first few sentences to condense the response
    important_sentences = ". ".join(full_summary.split(". ")[:3]) + "."  # First 3 sentences

    return important_sentences

# Define input parameters for the Wikipedia tool
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# Describe the tool's purpose
description = (
    "A tool that searches Wikipedia. "
    "Useful for answering general questions about people, places, companies, facts, historical events, or other subjects."
)

# Create a structured tool for Wikipedia search
wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia_search",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# Load the OpenAI API-based LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key="sk-RdqqgBgnocLR5BX5smKBEq21Wt4PtHjwf77ewKu64AT3BlbkFJx0fGc7GKZ36-k3Vs4Yo8nw-H-p6_ioTgpbY4jXkQgA")

# Define the system prompt and conversation style
template = """You are a helpful assistant specialized in searching information about destination locations.
{input}
{agent_scratchpad}"""
prompt = ChatPromptTemplate.from_template(template=template)

# Load the Wikipedia tool and create the agent
tools = [wiki_tool]
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Wikipedia Search Agent for destination location is ready.")

# Get input from the user and run the query
question = input("Enter destination location: ")
response = agent_executor.invoke({"input": question})
print(f"{response}")

Wikipedia Search Agent for destination location is ready.
Enter destination location: texas


> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'query': 'Texas'}`


Page: Texas
Summary: Texas (  TEK-səss, locally also  TEK-siz; Spanish: Texas or Tejas, pronounced [ˈtexas]) is the most populous state in the South Central region of the United States. It borders Louisiana to the east, Arkansas to the northeast, Oklahoma to the north, New Mexico to the west, and an international border with the Mexican states of Chihuahua, Coahuila, Nuevo León, and Tamaulipas to the south and southwest. Texas has a coastline on the Gulf of Mexico to the southeast.Texas is the most populous state in the South Central region of the United States. It borders Louisiana to the east, Arkansas to the northeast, Oklahoma to the north, New Mexico to the west, and has an international border with the Mexican states of Chihuahua, Coahuila, Nuevo León, and Tamaulipas to the south and southwest